In [1]:
import pandas as pd
import csv
from ast import literal_eval
import sys
import numpy as np
import glob
import regex as re
import seesus
from nltk.tokenize import sent_tokenize

/Users/mengcai/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
part1 = pd.read_csv("data/hpcc_text_part1_20240124.csv")
part2 = pd.read_csv("data/hpcc_text_part2_20240124.csv")
cities = pd.read_csv("data/smart_cities20240110.csv")

In [3]:
wps = pd.concat([part1, part2], ignore_index=True) # each row is a webpage

In [4]:
len(wps)

3265

In [ ]:
len(wps.GISJOIN.unique())

In [ ]:
len(cities[cities.smart==1])

In [ ]:
def paragraph_from_text(text, search_string):
    """Get paragraphs containing a keyword with its context"""
    split_text = text.lower().split("\n\n")
    found = False
    previous = ""
    para = []
    # to extract paragraphs containing a keyword and its immediate previous and after paragraphs
    for p in split_text:
        if found:
            para.append(p)
            found = False
        if p.find(search_string.lower()) != -1:
            para.append(previous)
            para.append(p)
            found = True
        else:
            found = False
        previous = p
    para = list(set(para)) # remove duplicates
    para_str = " ".join(para)
    return para_str

In [ ]:
short_text = []
for i in range(0, len(wps)):
    short = paragraph_from_text(wps.text.iloc[i], "smart")
    short_text.append(short)

In [ ]:
wps["mention"] = short_text

In [ ]:
for i in range(len(wps)):
    print(i)
    gisjoin = wps.GISJOIN.iloc[i]
    sents = sent_tokenize(wps.mention.iloc[i])
    for sent in sents:
        result = seesus.SeeSus(sent)
        sdg = result.sdg
        target = result.target
        see = result.see
        csv.writer(open(f"data/seesus_text/{gisjoin}.csv", "a")).writerow([sdg,target,see])